In [51]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import model_selection, preprocessing
from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import train_test_split

from sklearn import svm 
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [52]:
dt = pd.read_csv('facebook@buck.csv')

dt.head()


,travisJobId,testName,index,duration,count,failures,errors,skipped
0,155318303,//scripts:diff_rulekeys_test,0,0,12,0,0,0
1,155318303,//py/buck/zip:test,1,0,2,0,0,0
2,155318303,//programs:test,2,0,9,0,0,0
3,155318303,//src/com/facebook/buck/apple/project_generato...,3,0,3,0,0,0
4,155318303,//src/com/facebook/buck/apple/project_generato...,4,0,5,0,0,0


In [53]:
dt.tail()


,travisJobId,testName,index,duration,count,failures,errors,skipped
783985,93708442,com.facebook.buck.apple.AppleTestIntegrationTest,612,0,0,0,0,0
783986,93708442,com.facebook.buck.cli.InstallCommandIntegratio...,613,0,0,0,0,0
783987,93708442,com.facebook.buck.gwt.GwtBinaryIntegrationTest,614,27,1,0,0,0
783988,93708442,com.facebook.buck.jvm.java.JavaTestIntegration...,615,11,11,0,0,0
783989,93708442,com.facebook.buck.util.versioncontrol.HgCmdLin...,616,0,10,0,0,10


In [54]:
dt.shape

(783990, 8)

In [55]:
dt.describe()


,travisJobId,index,duration,count,failures,errors,skipped
count,7.839900e+05,783990.000000,783990.000000,783990.000000,783990.000000,783990.0,783990.000000
mean,1.202699e+08,345.344999,2.364779,5.933976,0.005094,0.0,0.295305
std,2.245196e+07,204.780485,10.756787,10.009804,0.229107,0.0,1.998691
min,3.424780e+07,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,9.935938e+07,170.000000,0.000000,2.000000,0.000000,0.0,0.000000
50%,1.115400e+08,342.000000,0.000000,3.000000,0.000000,0.0,0.000000
75%,1.440482e+08,513.000000,0.000000,6.000000,0.000000,0.0,0.000000
max,1.566613e+08,1297.000000,318.000000,210.000000,77.000000,0.0,81.000000


In [56]:
dt.columns

Index(['travisJobId', 'testName', 'index', 'duration', 'count', 'failures',
       'errors', 'skipped'],
      dtype='object')

In [57]:
dt.nunique()

travisJobId    1148
testName        894
index          1298
duration        250
count            84
failures         16
errors            1
skipped          32
dtype: int64

In [58]:
dt['index'].unique()

array([   0,    1,    2, ..., 1295, 1296, 1297])

In [59]:
dt['duration'].unique()

array([  0,   1,  23,   7,  19,   4,  24,  44,   2,  31,   5,   3, 139,
        32,  22,  38,   8,   6,  10,   9,  35,  16,  30,  11,  20,  28,
        21, 136,  39,  79,  14,  12,  89, 156,  15,  49,  37,  54,  13,
        74,  33,  25,  27,  17,  18,  77,  58,  71, 125,  51,  41,  43,
        42,  61,  70,  26,  83,  64, 122,  34,  59, 102,  99,  98,  73,
        76,  52,  82,  29,  93,  65, 120,  46,  50, 145, 223,  91, 108,
       179,  63, 130, 157,  48, 128, 100, 141,  68,  78,  36, 146,  45,
        96, 185, 126,  55, 163, 121, 114, 193,  72,  57,  67, 107, 143,
        40, 105, 158,  56,  86, 155, 103,  87,  66, 115,  47, 124, 123,
       116, 191, 206,  92,  85,  69, 271, 202,  53,  90,  97,  62, 109,
       142,  88, 177, 159, 134,  80, 148,  75, 111, 153, 106, 138, 117,
       160, 187,  60,  95, 172, 112, 212, 180, 110, 220, 150, 199, 149,
       144, 140, 104, 119, 161, 118,  81,  84, 135, 189, 113, 169, 154,
       186, 178, 166, 101, 137, 167, 210, 171, 226, 132, 133, 13

In [60]:
dt['failures'].unique()

array([ 0,  1,  6,  2,  5,  3,  9, 77, 17,  8, 11, 12,  7, 13, 24,  4])

In [61]:
binarized = preprocessing.Binarizer(threshold=1).transform(np.array(dt['failures']).reshape(-1,1))

dt['failures'].unique()



array([ 0,  1,  6,  2,  5,  3,  9, 77, 17,  8, 11, 12,  7, 13, 24,  4])

In [62]:
dt['failures'] = binarized

dt['failures'].unique()

array([0, 1])

In [63]:
#corelation = dt.corr()
#sns.heatmap(corelation, xticklabels = corelation.columns, yticklabels = corelation.columns, annot = True)
#sns.pairplot(dt)
#sns.distplot(data['failures'])
#sns.distplot(data['count'])
#sns.distplot(data['duration'])
#sns.catplot(x='failures', kind='box', data=data)

In [64]:
print(len(dt['testName'].unique()))

894


In [65]:
df=pd.DataFrame(dt)

ord_enc = OrdinalEncoder()
df["test_code"] = ord_enc.fit_transform(df[["testName"]])
df[["testName", "test_code"]].head(894)

df.head()

,travisJobId,testName,index,duration,count,failures,errors,skipped,test_code
0,155318303,//scripts:diff_rulekeys_test,0,0,12,0,0,0,7.0
1,155318303,//py/buck/zip:test,1,0,2,0,0,0,6.0
2,155318303,//programs:test,2,0,9,0,0,0,5.0
3,155318303,//src/com/facebook/buck/apple/project_generato...,3,0,3,0,0,0,9.0
4,155318303,//src/com/facebook/buck/apple/project_generato...,4,0,5,0,0,0,8.0


In [66]:
input_x = df.drop(['travisJobId','failures', 'testName'], axis = 1)
input_x.head()


,index,duration,count,errors,skipped,test_code
0,0,0,12,0,0,7.0
1,1,0,2,0,0,6.0
2,2,0,9,0,0,5.0
3,3,0,3,0,0,9.0
4,4,0,5,0,0,8.0


In [67]:
input_x.isnull().any()

index        False
duration     False
count        False
errors       False
skipped      False
test_code    False
dtype: bool

In [68]:
output_y = df['failures']
output_y.head()

0    0
1    0
2    0
3    0
4    0
Name: failures, dtype: int64

In [69]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(input_x, output_y, test_size=0.7)

#print(train_x, test_x, train_y, test_y)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)



(235197, 6) (548793, 6) (235197,) (548793,)


In [70]:
#Classifier1 
clf1 = GaussianNB()
clf1.fit(train_x, train_y)
predictions1 = clf1.predict(test_x)
print("\nAccuracy score ",accuracy_score(test_y, predictions1))
print("\nConfusion_matrix",confusion_matrix(test_y, predictions1))
print("\nClassification_report",classification_report(test_y, predictions1))
prob1 = clf1.predict_proba(test_x)
# Keeping only the positive class
prob1 = [p[1] for p in prob1]
print("\nROC score ",roc_auc_score(test_y, prob1))

df_proba1 = pd.DataFrame(clf1.predict_proba(test_x), columns=clf1.classes_)
df_out1 = pd.merge(df,df_proba1,how = 'left',left_index = True, right_index = True)
df_out1.head()

dfsort1 = df_out1.sort_values(by=[0,1])
print(dfsort1)

from google.colab import drive
drive.mount('/content/gdrive')

dfsort1.to_csv(r'/content/gdrive/MyDrive/Colab Notebooks/SQ/buck@nb.csv')


Accuracy score  0.9972157079263038

Confusion_matrix [[547263   1125]
 [   403      2]]

Classification_report               precision    recall  f1-score   support

           0       1.00      1.00      1.00    548388
           1       0.00      0.00      0.00       405

    accuracy                           1.00    548793
   macro avg       0.50      0.50      0.50    548793
weighted avg       1.00      1.00      1.00    548793


ROC score  0.8593612304057584
        travisJobId  ...    1
182969     97567763  ...  1.0
338174     99417189  ...  1.0
385649    146401478  ...  1.0
478052    150015809  ...  1.0
542465     96659724  ...  1.0
...             ...  ...  ...
783985     93708442  ...  NaN
783986     93708442  ...  NaN
783987     93708442  ...  NaN
783988     93708442  ...  NaN
783989     93708442  ...  NaN

[783990 rows x 11 columns]
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [71]:
#Classifier2
clf2 = LogisticRegression()
clf2.fit(train_x, train_y)
predictions2 = clf2.predict(test_x)
print("\nAccuracy score ",accuracy_score(test_y, predictions2))
print("\nConfusion_matrix",confusion_matrix(test_y, predictions2))
print("\nClassification_report",classification_report(test_y, predictions2))
prob2 = clf2.predict_proba(test_x)
# Keeping only the positive class
prob2 = [p[1] for p in prob2]
print("\nROC score ",roc_auc_score(test_y, prob2))

df_proba2 = pd.DataFrame(clf2.predict_proba(test_x), columns=clf2.classes_)
df_out2 = pd.merge(df,df_proba2,how = 'left',left_index = True, right_index = True)
df_out2.head()

dfsort2 = df_out2.sort_values(by=[0,1])
print(dfsort2)

#from google.colab import drive
#drive.mount('/content/gdrive')

dfsort2.to_csv(r'/content/gdrive/MyDrive/Colab Notebooks/SQ/buck@lr.csv')



Accuracy score  0.9992620168260163

Confusion_matrix [[548388      0]
 [   405      0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Classification_report               precision    recall  f1-score   support

           0       1.00      1.00      1.00    548388
           1       0.00      0.00      0.00       405

    accuracy                           1.00    548793
   macro avg       0.50      0.50      0.50    548793
weighted avg       1.00      1.00      1.00    548793


ROC score  0.8360114204982558
        travisJobId  ...         1
414697     94507305  ...  0.232208
362687     99479962  ...  0.202138
188035    142184827  ...  0.183873
119264    111795017  ...  0.166876
351809    111112617  ...  0.157366
...             ...  ...       ...
783985     93708442  ...       NaN
783986     93708442  ...       NaN
783987     93708442  ...       NaN
783988     93708442  ...       NaN
783989     93708442  ...       NaN

[783990 rows x 11 columns]


In [72]:
#Classifier3
clf3 = RandomForestClassifier()
clf3.fit(train_x, train_y)
predictions3 = clf3.predict(test_x)
print("\nAccuracy score ",accuracy_score(test_y, predictions3))
print("\nConfusion_matrix",confusion_matrix(test_y, predictions3))
print("\nClassification_report",classification_report(test_y, predictions3))
prob3 = clf3.predict_proba(test_x)
# Keeping only the positive class
prob3 = [p[1] for p in prob3]
print("\nROC score ",roc_auc_score(test_y, prob3))

df_proba3 = pd.DataFrame(clf3.predict_proba(test_x), columns=clf3.classes_)
df_out3 = pd.merge(df,df_proba3,how = 'left',left_index = True, right_index = True)
df_out3.head()

dfsort3 = df_out3.sort_values(by=[0,1])
print(dfsort3)

dfsort3.to_csv(r'/content/gdrive/MyDrive/Colab Notebooks/SQ/buck@rf.csv')


Accuracy score  0.9997193841758185

Confusion_matrix [[548346     42]
 [   112    293]]

Classification_report               precision    recall  f1-score   support

           0       1.00      1.00      1.00    548388
           1       0.87      0.72      0.79       405

    accuracy                           1.00    548793
   macro avg       0.94      0.86      0.90    548793
weighted avg       1.00      1.00      1.00    548793


ROC score  0.9563946703681102
        travisJobId  ...    1
10555      97002283  ...  1.0
14228     110207956  ...  1.0
16624     154173184  ...  1.0
18317     154706739  ...  1.0
20892     101372449  ...  1.0
...             ...  ...  ...
783985     93708442  ...  NaN
783986     93708442  ...  NaN
783987     93708442  ...  NaN
783988     93708442  ...  NaN
783989     93708442  ...  NaN

[783990 rows x 11 columns]
